In [2]:
STREAM_NAME = "TEST"

In [3]:
from jsm.api.client import Client as JS

In [4]:
js = JS()
await js.connect()

- Create a new stream

In [6]:
response = await js.stream_create("TEST", subjects=["test.>"], max_consumers=10, raise_on_error=True)
response.config.dict()

{'name': 'TEST',
 'subjects': ['test.>'],
 'retention': <Retention.limits: 'limits'>,
 'max_consumers': 10,
 'max_msgs': -1,
 'max_msgs_per_subject': -1,
 'max_bytes': -1,
 'max_age': 0,
 'max_msg_size': -1,
 'storage': <Storage.file: 'file'>,
 'num_replicas': 1,
 'no_ack': False,
 'template_owner': None,
 'discard': <Discard.old: 'old'>,
 'duplicate_window': 120000000000,
 'placement': None,
 'mirror': None,
 'sources': None}

- List stream names

In [7]:
response = await js.stream_names()
response.streams

['TEST']

- List streams

In [8]:
response = await js.stream_list()

In [9]:
response.streams[0].dict()

{'config': {'name': 'TEST',
  'subjects': ['test.>'],
  'retention': <Retention.limits: 'limits'>,
  'max_consumers': 10,
  'max_msgs': -1,
  'max_msgs_per_subject': -1,
  'max_bytes': -1,
  'max_age': 0,
  'max_msg_size': -1,
  'storage': <Storage.file: 'file'>,
  'num_replicas': 1,
  'no_ack': False,
  'template_owner': None,
  'discard': <Discard.old: 'old'>,
  'duplicate_window': 120000000000,
  'placement': None,
  'mirror': None,
  'sources': None},
 'state': {'messages': 0,
  'bytes': 0,
  'first_seq': 0,
  'first_ts': '0001-01-01T00:00:00Z',
  'last_seq': 0,
  'last_ts': '0001-01-01T00:00:00Z',
  'deleted': None,
  'num_deleted': None,
  'lost': None,
  'consumer_count': 0},
 'created': '2021-07-25T23:18:17.0245932Z',
 'cluster': None,
 'mirror': None,
 'sources': None}

- Get stream info

In [8]:
response = await js.stream_info(STREAM_NAME)

In [9]:
response.dict()

{'type': 'io.nats.jetstream.api.v1.stream_info_response',
 'config': {'name': 'TEST',
  'subjects': ['test.>'],
  'retention': <Retention.limits: 'limits'>,
  'max_consumers': 10,
  'max_msgs': -1,
  'max_msgs_per_subject': -1,
  'max_bytes': -1,
  'max_age': 0,
  'max_msg_size': -1,
  'storage': <Storage.file: 'file'>,
  'num_replicas': 1,
  'no_ack': False,
  'template_owner': None,
  'discard': <Discard.old: 'old'>,
  'duplicate_window': 120000000000,
  'placement': None,
  'mirror': None,
  'sources': None},
 'state': {'messages': 0,
  'bytes': 0,
  'first_seq': 0,
  'first_ts': '0001-01-01T00:00:00Z',
  'last_seq': 0,
  'last_ts': '0001-01-01T00:00:00Z',
  'deleted': None,
  'num_deleted': None,
  'lost': None,
  'consumer_count': 0},
 'created': '2021-07-25T22:57:18.9843268Z',
 'cluster': None,
 'mirror': None,
 'sources': None}

- By defaults errors are not raised, but it can be easily changed using the `raise_on_error` keyword argument:

In [10]:
await js.stream_info("toto")

IoNatsJetstreamApiV1ErrorResponse(type='io.nats.jetstream.api.v1.stream_info_response', error=IoNatsJetstreamApiV1ErrorItem(code=404, description='stream not found'))

In [12]:
from jsm.models.errors import IoNatsJetstreamApiV1Exception
from loguru import logger


try:
    await js.stream_info("toto", raise_on_error=True)
except IoNatsJetstreamApiV1Exception as err:
    logger.error(repr(err))

2021-07-26 00:57:33.271 | ERROR    | __main__:<module>:8 - IoNatsJetstreamApiV1Exception('Status code=404 | Description=stream not found | Type=io.nats.jetstream.api.v1.stream_info_response')


- Update a stream

In [13]:
response = await js.stream_update(STREAM_NAME, subjects=["test.>", "demo.>"])

In [14]:
response.config

Config(name='TEST', subjects=['test.>', 'demo.>'], retention=<Retention.limits: 'limits'>, max_consumers=10, max_msgs=-1, max_msgs_per_subject=-1, max_bytes=-1, max_age=0, max_msg_size=-1, storage=<Storage.file: 'file'>, num_replicas=1, no_ack=False, template_owner=None, discard=<Discard.old: 'old'>, duplicate_window=120000000000, placement=None, mirror=None, sources=None)

Checking that configuration was indeed updated can be done easily:

In [15]:
assert (await js.stream_info(STREAM_NAME)).config.subjects == ["test.>", "demo.>"]

- Delete a stream

In [16]:
await js.stream_delete(STREAM_NAME)

IoNatsJetstreamApiV1StreamDeleteResponse(type='io.nats.jetstream.api.v1.stream_delete_response', success=True)

- Recreate the stream

In [17]:
await js.stream_create(STREAM_NAME, subjects=["test.>"])

IoNatsJetstreamApiV1StreamCreateResponse(type='io.nats.jetstream.api.v1.stream_create_response', config=Config(name='TEST', subjects=['test.>'], retention=<Retention.limits: 'limits'>, max_consumers=-1, max_msgs=-1, max_msgs_per_subject=-1, max_bytes=-1, max_age=0, max_msg_size=-1, storage=<Storage.file: 'file'>, num_replicas=1, no_ack=False, template_owner=None, discard=<Discard.old: 'old'>, duplicate_window=120000000000, placement=None, mirror=None, sources=None), state=State(messages=0, bytes=0, first_seq=0, first_ts='0001-01-01T00:00:00Z', last_seq=0, last_ts='0001-01-01T00:00:00Z', deleted=None, num_deleted=None, lost=None, consumer_count=0), created='2021-07-25T22:57:48.6629514Z', cluster=None, mirror=None, sources=None)

- Publish a message to the stream

In [18]:
await js.publish("test.demo", b"hello world")

- Check the stream state

In [19]:
response = await js.stream_info(STREAM_NAME)
response.state

State(messages=1, bytes=50, first_seq=1, first_ts='2021-07-25T22:57:50.364609Z', last_seq=1, last_ts='2021-07-25T22:57:50.364609Z', deleted=None, num_deleted=None, lost=None, consumer_count=0)

We can see that there is 1 message, with `first_seq` being 1 and `last_seq` being also 1. We can fetch a message by its sequence:

In [20]:
response = await js.stream_msg_get(STREAM_NAME, 1)

In [21]:
response.message

Message(subject='test.demo', seq=1, data='aGVsbG8gd29ybGQ=', time='2021-07-25T22:57:50.364609Z', hdrs=None)

- Delete a message

In [22]:
response = await js.stream_msg_delete(STREAM_NAME, 1)

In [23]:
response.success

True

- Check the stream state again

In [24]:
response = await js.stream_info(STREAM_NAME)

In [25]:
response.state

State(messages=0, bytes=0, first_seq=2, first_ts='1970-01-01T00:00:00Z', last_seq=1, last_ts='2021-07-25T22:57:50.364609Z', deleted=None, num_deleted=None, lost=None, consumer_count=0)

As you can see, the `first_seq` is now 2 because we deleted the previous first message. Attempting to fetch a message not received yet will lead to a Jetstream error:

In [26]:
await js.stream_msg_get(STREAM_NAME, 2, raise_on_error=True)

IoNatsJetstreamApiV1Exception: Status code=404 | Description=no message found | Type=io.nats.jetstream.api.v1.stream_msg_get_response

- Let's publish a new message to the stream

In [27]:
await js.publish("test.demo", b"hello again")

In [28]:
response = await js.stream_info(STREAM_NAME)
response.state

State(messages=1, bytes=50, first_seq=2, first_ts='2021-07-25T22:58:03.3321065Z', last_seq=2, last_ts='2021-07-25T22:58:03.3321065Z', deleted=None, num_deleted=None, lost=None, consumer_count=0)

- The second message is now available

In [29]:
await js.stream_msg_get(STREAM_NAME, 2)

IoNatsJetstreamApiV1StreamMsgGetResponse(type='io.nats.jetstream.api.v1.stream_msg_get_response', message=Message(subject='test.demo', seq=2, data='aGVsbG8gYWdhaW4=', time='2021-07-25T22:58:03.3321065Z', hdrs=None))

- Let's purge the stream

In [30]:
await js.stream_purge(STREAM_NAME)

IoNatsJetstreamApiV1StreamPurgeResponse(type='io.nats.jetstream.api.v1.stream_purge_response', success=True, purged=1)

- And fetch the stream info one last time

In [31]:
response = await js.stream_info(STREAM_NAME)
response.state

State(messages=0, bytes=0, first_seq=3, first_ts='0001-01-01T00:00:00Z', last_seq=2, last_ts='2021-07-25T22:58:03.3321065Z', deleted=None, num_deleted=None, lost=None, consumer_count=0)

`first_ts` is reinitialized but `last_ts` is kept

- Backup a whole stream

TODO